## Imports

In [1]:
import sys
import os
import numpy as np
import numpy.lib.recfunctions as nprf
from sklearn import linear_model
import matplotlib as mpl
from scipy.stats import gaussian_kde
from importlib import reload
import pandas as pd

mpl.use('Agg')

mpl.rcParams['figure.figsize'] = (7,7)
mpl.rcParams['figure.facecolor'] = (1,1,1,1)
mpl.rcParams.update({'font.size':22})

% matplotlib inline

import matplotlib.pyplot as plt

In [2]:
import tools.matt_tools as matt
import tools.cnn_tools as cnn

reload(matt)
reload(cnn)

<module 'tools.cnn_tools' from '/home/mho1/halo_cnn/notebooks/tools/cnn_tools.py'>

In [3]:
wdir = '/home/mho1/scratch/halo_cnn'

# Load

In [4]:
data_file = os.path.join(wdir,
                         'raw',
                         'MDPL2_Rockstar_z=0.117_Macc=1e11.csv')

In [5]:
dat_MD = pd.read_csv(data_file)

In [6]:
dat_MD.head()

,row_id,rockstarId,upId,M200b,Rvir,x,y,z,vx,vy,vz,M500c,Rs,Macc,Vacc,Mvir,M200c
0,1,11830268795,-1,1.716200e+11,115.780,0.24644,0.24207,267.242,56.51,-380.49,-149.36,1.219400e+11,12.480,1.625800e+11,98.51,1.625800e+11,1.475300e+11
1,2,11821679825,11821679827,2.604400e+11,134.682,0.80576,1.62425,283.605,63.22,132.57,113.04,1.656000e+11,8.144,2.950600e+11,124.31,2.559200e+11,2.137700e+11
2,3,11821797355,-1,3.025900e+11,139.524,0.05414,1.74925,460.638,344.27,176.85,-145.34,1.972100e+11,11.197,2.845200e+11,121.28,2.845200e+11,2.423700e+11
3,4,11821801845,-1,1.896800e+11,116.491,0.46209,1.73137,470.666,253.49,-31.45,-230.78,1.144100e+11,13.009,1.656000e+11,102.83,1.656000e+11,1.490400e+11
4,5,11821864805,-1,4.365700e+11,152.337,0.49521,1.10364,564.098,253.09,264.77,217.72,2.438800e+11,12.862,3.703300e+11,134.21,3.703300e+11,3.071000e+11


# Scratch

In [2]:
from collections import OrderedDict

In [4]:
par = OrderedDict()

In [5]:
par = OrderedDict([
    ('wdir', '/home/mho1/scratch/halo_cnn/'),
    ('model_name', 'halo_cnn1d_r')
])

In [13]:
for key in par.keys():
    print(key + ' : ' + str(par[key]))

wdir : /home/mho1/scratch/halo_cnn/
model_name : halo_cnn1d_r
